# 1. 필요한 패키지 Install & Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''! pip install -U accelerate
! pip install -U transformers'''

import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.39.3', '0.29.2')

In [ ]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, BitsAndBytesConfig
from tqdm.notebook import tqdm
import accelerate
import transformers

# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# 2. 허깅페이스에서 KOBART Model & Tokenizer Load

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd

# google/flan-t5-base도 시도해볼것

# 모델과 토크나이저 로드
model_name = "gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

# 3. 데이터 전처리

In [ ]:
## 데이터 로드
train_data = pd.read_csv('./drive/MyDrive/제주어번역모델생성/데이터/space-2773763.csv')

In [ ]:
## 결측치 제거
train_data = train_data.loc[train_data.standard_form.notnull()]

## 제주어 표준어 구분
## 이거 사실 안해도 되는데 True False 보다는 string 이 나을거 같아서 그냥 변환
train_data.isDialect = train_data.isDialect.apply(lambda x : '제주' if x == True else '표준')

## 발화 길이가 3 이하인 발화 제거
## "3 이하" 는 변경 가능
train_data['form_len'] = train_data.standard_form.apply(lambda x : len(str(x)))
train_data = train_data.loc[train_data.form_len > 3]

## 제주 발화만 남기기
## 사실 isDialect가 표준인 경우에는 두 문장이 모두 표준어이기 때문에 source, target 문장이 같아서 의미가 없다고 생각해 제거
train_data = train_data.loc[train_data.isDialect == '제주']

## form에 # 있는 경우 제거
train_data.form_샵 = train_data.form.apply(lambda x : 1 if '#' in x else 0)
train_data = train_data.loc[train_data.form_샵 == 0]

## form에 & 있는 경우 제거
train_data.form_앤드 = train_data.form.apply(lambda x : 1 if '&' in x else 0)
train_data = train_data.loc[train_data.form_앤드 == 0]

## 최대 길이 64로 지정
## 아래에서 데이터 포맷팅하는데 max_length가 64기 때문에 실행 but, max_length를 늘린다면 이 작업은 그에 맞춰서 해줘도 된다
train_data = train_data.loc[train_data.form_len <= 64]

In [ ]:
## 필자의 경우에는 데이터가 너무 많아서 용량 초과되는 문제가 발생..
## 용량이 충분하다면 모든 train_data를 사용해도 무관
train_data = train_data.head(700000)

# 4. 데이터 포맷팅 및 토크나이징

In [ ]:
## 제주어 토큰, 표준어 토큰 정의하기
jeju_token = "[제주]"
standard_token = "[표준]"

## 양방향 데이터 리스트 생성
bidirectional_data = []

for dialect, standard in zip(train_data['dialect_form'], train_data['standard_form']):
    ## 토큰이 [제주] 일 경우 제주어 -> 표준어
    bidirectional_data.append({
        "source": jeju_token + " " + dialect,
        "target": standard
    })
    ## 토큰이 [표준] 일 경우 표준어 -> 제주어
    bidirectional_data.append({
        "source": standard_token + " " + standard,
        "target": dialect
    })

## 데이터 토크나이징
tokenized_data = []
for item in bidirectional_data:
    source_encodings = tokenizer(item['source'], max_length=64, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(item['target'], max_length=64, truncation=True, padding="max_length", return_tensors="pt")
    tokenized_data.append({
        "input_ids": source_encodings["input_ids"],
        "attention_mask": source_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    })

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
## 데이터가 어떻게 토큰화되었는지 한번 확인해보슈
tokenized_data[:2]

# 5. Traning

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

## 변환된 데이터를 DataFrame으로 변환
formatted_data_df = pd.DataFrame([{
    "input_ids": fd["input_ids"].numpy().tolist()[0],  ## Tensor를 리스트로 변환
    "attention_mask": fd["attention_mask"].numpy().tolist()[0],  ## Tensor를 리스트로 변환
    "labels": fd["labels"].numpy().tolist()[0]  ## Tensor를 리스트로 변환
} for fd in tokenized_data])

## 데이터를 Dataset 형식으로 변환
train_dataset = Dataset.from_pandas(formatted_data_df)

In [ ]:
## 데이터셋 한번 확인해보슈
train_dataset

In [ ]:
## 학습 데이터셋을 학습 및 평가용으로 분리 (예: 90% 학습, 10% 평가)
train_test_split = train_dataset.train_test_split(test_size=0.1)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

## 학습 매개변수 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="./drive/MyDrive/제주어번역모델생성/",    ## 결과물을 저장할 디렉토리
    evaluation_strategy="epoch",                         ## 평가 전략
    learning_rate=2e-5,                                  ## 학습률
    per_device_train_batch_size=16,                      ## 디바이스 당 배치 크기
    weight_decay=0.01,                                   ## 가중치 감소
    save_total_limit=3,                                  # 저장할 최대 체크포인트 수
    num_train_epochs=3,                                  # 학습 에폭 수
    predict_with_generate=True,                          # 생성을 사용한 예측 활성화
)

In [ ]:
## 학습 준비
trainer = Seq2SeqTrainer(
    model=model,                         ## 학습할 모델
    args=training_args,                  ## 학습 설정
    train_dataset=dataset_dict['train'], ## 학습 데이터셋
    eval_dataset=dataset_dict['test'],   ## 평가 데이터셋
    tokenizer=tokenizer
)

## 학습 시작
trainer.train()